1. Intro & Setup

In [ ]:
!pip install -r requirements.txt

2. (Optional) Download a subset of images for demo

In [ ]:
%env DATA_DIR=./data
!python scripts/download_images.py  # or set num_images inside script

3. Build embeddings & FAISS index

In [ ]:
!python scripts/build_index.py

4. Smoke test the API locally (non-Docker)

In [ ]:
import requests, json, threading, time, uvicorn
from app.main import app

def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="warning")

thr = threading.Thread(target=run_app, daemon=True)
thr.start()
time.sleep(2)

resp = requests.post("http://localhost:8000/search",
                     json={"query":"red car on street", "top_k":5})
resp.json()


5. Visualize results (inline)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import requests

results = resp.json()["results"]
plt.figure(figsize=(12,6))
for i, r in enumerate(results, 1):
    plt.subplot(1, len(results), i)
    img = Image.open(r["path"]).convert("RGB")
    plt.imshow(img)
    plt.title(f"{i}. {r['similarity']:.2f}")
    plt.axis('off')
plt.show()

for r in results:
    print(f"{r['rank']}. {r['path']} -> {r['explanation']}")
